# Stacks and queues
A stack has a last in first out structure (LIFO), like a can of pringles. A queue has a first in first out (FIFO) structure. Appending to and popping from are standard with dynamic arrays in python and use $O(1)$ amortized time. A queue struction may require use of the deque collection, as inserting at the start of a dynamic array requires $O(n)$ time.

#### Compress array
Given an array of integers, perform (repeatedly) a compress operation that finds the first consecutive equal numbers and combines them as their sum.

In [3]:
# Rather than constantly recomposing the array (with each alteration taking O(n) time), we create a stack. 
# This allows us to add to or remove from the end with O(n) time. Space complexity is O(n).

def compress_array(arr):
    n = len(arr)
    if n < 2:
        return arr
    stack = []
    i = 0
    while i < len(arr):
        if stack == []:
            stack.append(arr[i])
            i += 1
        elif stack[-1] == arr[i]:
            stack.pop()
            arr[i] += arr[i]
        else:
            stack.append(arr[i])
            i += 1
    return stack
            

In [4]:
arr = [8,4,2,2,2,4]
print(f"Compress {arr}: {compress_array(arr)}")
arr = [4,4,4,4]
print(f"Compress {arr}: {compress_array(arr)}")
arr = [1,2,3,4]
print(f"Compress {arr}: {compress_array(arr)}")

Compress [8, 4, 2, 2, 2, 4]: [16, 2, 4]
Compress [4, 4, 4, 4]: [16]
Compress [1, 2, 3, 4]: [1, 2, 3, 4]


In [5]:
# A more elegant solution...
def compress_array(arr):
    stack = []
    for num in arr:
        while stack and stack[-1] == num:
            num += stack.pop()
        stack.append(num)
    return stack

#### Compress array by k
Given an array of integers and k >= 2, run compress operations repeatedly on the first block of k-repeated entries, each time replacing them with their sum.

In [7]:
def compress_by_k(arr, k):
    if k < 2:
        raise ValueError("k must be >= 2")
    if len(arr) < k:
        return arr
    stack = []
    for num in arr:
        while len(stack) >= k-1 and stack[-k+1:] == [num]*(k-1):
            for i in range(k-1):
                num += stack.pop()
        stack.append(num)
    return stack

In [8]:
arr = [1,9,9,3,3,3,4]
k = 3
print(f"Compress {arr} by {k}: {compress_by_k(arr, k)}")
arr = [8,4,2,2]
k = 2
print(f"Compress {arr} by {k}: {compress_by_k(arr, k)}")
arr = [4,4,4,4]
k = 5
print(f"Compress {arr} by {k}: {compress_by_k(arr, k)}")

Compress [1, 9, 9, 3, 3, 3, 4] by 3: [1, 27, 4]
Compress [8, 4, 2, 2] by 2: [16]
Compress [4, 4, 4, 4] by 5: [4, 4, 4, 4]


In [9]:
# An alternative approach involves using an array within stack to record how many times a number has occurred. 
# However, this solution is more cumbersome and requires additional and space complexity as an additional array is created.
def compress_by_k(arr, k):
    stack = []
    def merge(num):
        if not stack or stack[-1][0] != num:
            stack.append([num, 1])
        elif stack[-1][1] < k-1:
            stack[-1][1] += 1
        else:
            stack.pop()
            merge(num * k)
    for num in arr:
        merge(num)
    result = []
    for num, count in stack:
        result += [num] * count
    return result

#### Viewer counter class
Implement a `ViewerCounter` class that tracks viewer counts within a configurable time window for a live-stream event. Viewer types may be 'guest', 'follower' and 'subscriber'.

In [11]:
# Dynamic arrays in python cannot be used efficiently to implement queue structures, as you can't pop efficiently from the front of them.
# As such, this code uses a generic Queue() api.

class ViewerCounter:
    def __init__(self, window):
        self.queues = {'guest': Queue(), 'follower': Queue(), 'subscriber': Queue()}
        self.window = window
        if window < 1:
            raise IndexError('Window size must be >= 1')
    def join(self, t, v):
        self.queues[v].push(t)
    def get_viewers(self, t, v):
        queue = self.queues[v]
        while not queue.empty() and queue.peek() < t - self.window:
            queue.pop() # this removes all viewers from before the time window starts
        return queue.size()

#### Current URL
Implement back arrow browser functionality using an array of actions undertaken by a user. Each element in actions has two elements: go or back, and a URL string (if go) or a number $\ge 1$ denoting the number of times the user goes back. The first action is always go; the URL stays the same if there are no previous ones. Return the URL after all actions have been performed.

In [13]:
def current_url(actions):
    stack = []
    for action, value in actions:
        if action == 'go':
            stack.append(value)
        else:
            for v in range(value):
                if len(stack) ==1:
                    break
                else:
                    stack.pop()
    return stack.pop()
                

In [14]:
actions = [['go', 'google.com'], ['go', 'wikipedia.com'], ['go', 'amazon.com'], ['back', 1], 
           ['go', 'youtube.com'], ['go', 'netflix.com'], ['back', 2]]

current_url(actions)

'wikipedia.com'

#### Current URL with forward
Now add 'forward' functionality, which reverses the back functionality. Note that a 'go' instruction clears the option to go forward and going forward past the most recent page does nothing.

In [16]:
def current_url_forward(actions):
    stack = []
    back_stack = []
    for action, value in actions:
        if action == 'go':
            stack.append(value)
            back_stack = []
        elif action == 'back':
            while value > 0 and len(stack) > 1:
                back_stack.append(stack.pop())
                value -= 1    
        else:
            while value > 0 and len(back_stack) > 0:
                stack.append(back_stack.pop())
                value -= 1
    return stack.pop()

In [17]:
actions = [['go', 'google.com'], ['go', 'wikipedia.com'], ['back', 1], ['forward', 1], 
           ['back', 3], ['go', 'netflix.com'], ['forward', 3]]

current_url_forward(actions)

'netflix.com'

#### Balanced partition
Given a balanced string of parentheses, return the number of balanced substrings it contains.

In [19]:
def balanced_partition(s):
    depth = 0
    res = 0
    for c in s:
        if c == '(':
            depth += 1
        else:
            depth -= 1
            if depth == 0:
                res += 1
    return res

In [20]:
s = "((()))(()())()(()(())))"
balanced_partition(s)

4

#### Custom brackets
You are given a string and an array of bracket pairs. Return whether the string is balanced in terms of the bracket pairs.

In [22]:
def custom_brackets(s, arr):
    brack_stack = []
    for c in s:
        for bracket in arr:
            if c == bracket[0]:
                brack_stack.append(c)
            elif c == bracket[1]:
                if not brack_stack or brack_stack.pop() != bracket[0]:
                    return False
    return not brack_stack
# Note this solution has time complexity O(s*n). A slightly more efficient result could be achieved 
# by creating a a dictionary of opening brackets and set of closed brackets. There is a trade-off here with space complexity. 
# In any case, n is unlikely to be large.

In [23]:
s = "((a+b)*[c-d]-{e/f})"
brackets = ["()", "[]", "{}"]
print(f"Checking for these balanced bracket pairs - {brackets} - in '{s}': {custom_brackets(s, brackets)}")

s = "()[}"
brackets = ["()", "[]", "{}"]
print(f"Checking for these balanced bracket pairs - {brackets} - in '{s}': {custom_brackets(s, brackets)}")

s = "([)]"
brackets = ["()", "[]", "{}"]
print(f"Checking for these balanced bracket pairs - {brackets} - in '{s}': {custom_brackets(s, brackets)}")

s = ",div. hello :) </div>"
brackets = ["<>", "()"]
print(f"Checking for these balanced bracket pairs - {brackets} - in '{s}': {custom_brackets(s, brackets)}")

Checking for these balanced bracket pairs - ['()', '[]', '{}'] - in '((a+b)*[c-d]-{e/f})': True
Checking for these balanced bracket pairs - ['()', '[]', '{}'] - in '()[}': False
Checking for these balanced bracket pairs - ['()', '[]', '{}'] - in '([)]': False
Checking for these balanced bracket pairs - ['<>', '()'] - in ',div. hello :) </div>': False


#### Longest balanced subsequence
Given a string of parentheses, return the longest balanced substring.

In [45]:
def longest_balanced_substring(s):
    stack = []
    removals = set()
    for i, c in enumerate(s):
        if c == "(":
            stack.append(i)
        elif not stack: 
            removals.add(i) # This will remove any unbalaced closing parentheses
        else:
            stack.pop()
    # The stack now contains the indices of any unbalanced opening parentheses - the following will remove them from the string
    for i in stack:
        removals.add(i)
    substring = ''.join([s[i] for i in range(len(s)) if i not in removals])
    return substring
        

In [47]:
s = "))(())(()"
print(f"Retrieving longest balanced subsequence from '{s}': {longest_balanced_substring(s)}")
s = "(()(()("
print(f"Retrieving longest balanced subsequence from '{s}': {longest_balanced_substring(s)}")
s = "())(()"
print(f"Retrieving longest balanced subsequence from '{s}': {longest_balanced_substring(s)}")
s = "("
print(f"Retrieving longest balanced subsequence from '{s}': {longest_balanced_substring(s)}")

Retrieving longest balanced subsequence from '))(())(()': (())()
Retrieving longest balanced subsequence from '(()(()(': ()()
Retrieving longest balanced subsequence from '())(()': ()()
Retrieving longest balanced subsequence from '(': 
